In [5]:
import numpy as np
#Load the predicted 9x12 array

#1st pass
im1=np.array([[4,4,4,4,4,4,4,4,4,4,4,4],
              [6,6,2,1,6,6,6,6,6,1,1,2],
              [6,6,6,1,1,6,6,6,6,1,1,2],
              [2,6,6,6,1,5,5,5,6,1,1,2],
              [5,6,6,6,5,5,5,5,5,1,5,5],
              [5,5,2,5,5,5,5,5,5,1,5,5],
              [5,5,2,5,5,5,5,5,5,6,5,5],
              [2,6,6,6,5,5,5,5,5,6,2,2],
              [2,6,6,6,6,6,6,2,2,6,2,2]])

#zoomed into driveway
im2=np.array([[2,2,2,1,1,1,2,6,6,6,6,6],
              [2,2,2,1,1,1,2,6,6,6,6,6],
              [2,2,2,1,1,1,2,6,6,6,6,6],
              [2,2,2,1,1,1,1,6,6,6,6,6],
              [2,2,2,6,1,1,1,6,6,6,6,6],
              [6,6,6,6,1,1,1,1,6,6,6,6],
              [6,6,6,6,6,1,1,1,6,6,6,6],
              [6,6,6,6,6,6,1,1,2,2,2,2],
              [6,6,6,6,6,6,6,1,5,5,5,5]])

In [6]:
#%%timeit
from scipy.ndimage.measurements import label
from scipy.ndimage.measurements import center_of_mass

A=im1

#Center of the 9x12 array
img_center=np.array([4,5.5])

#Label all the driveways and roofs
driveway, num_driveway = label(A==1)
roof, num_roof = label(A==5)

#Save number of driveways into array
d=np.arange(1,num_driveway+1)
r=np.arange(1,num_roof+1)

#Find the center of the all the driveways
driveway_center=center_of_mass(A,driveway,d)
roof_center=center_of_mass(A,roof,r)

print(driveway_center)

#Function to find the closest roof/driveway
def closest(list,img_center):
    closest=list[0]
    for c in list:
        if np.linalg.norm(c-img_center) < np.linalg.norm(closest-img_center):
            closest = c
    return closest

#Find the closest roof to the center of the image
closest_roof=closest(roof_center,img_center)

#Find the closest driveway to the closest roof
closest_driveway=closest(driveway_center,np.asarray(closest_roof))
print(closest_driveway)

[(2.0, 3.5), (2.625, 9.375)]
(2.0, 3.5)


In [8]:
#Look for 3x3 driveway when we have reached a certain height (maybe 5m above ground)
a=im2

#Sliding window function
def sliding_window_view(arr, shape):
    n = np.array(arr.shape) 
    o = n - shape + 1 # output shape
    strides = arr.strides
    
    new_shape = np.concatenate((o, shape), axis=0)
    new_strides = np.concatenate((strides, strides), axis=0)
    return np.lib.stride_tricks.as_strided(arr ,new_shape, new_strides)

#Creates a 7x10 ndarray with all the 3x3 submatrices
sub_image=sliding_window_view(a,(3,3))

#Empty list
driveway_list=[]

#Loop through the 7x10 ndarray
for i in range(0,7):
    for j in range(i,10):
        #Calculate the total of the  submatrices
        output=sum(sum(sub_image[i,j]))
        #if the output is 9, that means we have a 3x3 that is all driveway
        if output==9:
            #append the i(row) and j(column) to a list declared previously
            #we add 1 to the i and j to find the center of the 3x3
            driveway_list.append((i+1,j+1))
            
#Call closest function to find driveway closest to house.             
closest_driveway=closest(driveway_list,np.asarray(closest_roof))
print(closest_driveway)


(4, 5)


In [65]:
#Read altitude from csv & Ground Sampling
import csv

def GSD(alt):
    sensor_height=4.5 #mm
    sensor_width=6.17 #mm
    focal_length=1.8
    image_height=1080 #pixels
    image_width=1920 #pixels

    #GSD = (sensor height (mm) x flight height (m) x 100) / (focal lenght (mm) x image height (pixel))
    GSD_x=((sensor_width*altitude*100)/(focal_length*image_width))
    GSD_y=((sensor_height*altitude*100)/(focal_length*image_height))
    return (GSD_x,GSD_y)

#Read alt.csv
with open('alt.csv', 'r') as csvfile:
    alt_list = [line.rstrip('\n') for line in csvfile]
        
#chose last value in alt_list        
altitude=int(alt_list[-1]) #in meters
multiplier=GSD(altitude) #cm/pixel
move_coordinates=np.asarray(closest_driveway)*np.asarray(multiplier)*40 #40 is the center of the 80x80 superpixel
print(closest_driveway)
print(multiplier)
print(move_coordinates)

(4, 5)
(1.7853009259259258, 2.314814814814815)
[285.64814815 462.96296296]


In [239]:
# Write to CSV
import csv
with open('coordinates.csv', 'a', newline='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',')
    filewriter.writerow(move_coordinates)